In [8]:
import pandas as pd
import numpy as np
import datetime
import time
import os

종목토론방 date tagging

In [2]:
df_board = pd.read_csv('./../Data/boardcomment/boardcomment_kia.csv',dtype={'com_code': np.str,'comment': np.str})
df_date = pd.read_csv('./../Data/주가_휴장일추출.csv')

C:\Users\seon\AppData\Local\Temp\ipykernel_25176\1908212109.py:1: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df = pd.read_csv('./../Data/boardcomment/boardcomment_kia.csv',dtype={'com_code': np.str,'comment': np.str})


In [3]:
df_board.rename(columns={'com_code':'code','comment':'content','decommend':'unlike','recommend':'like','view':'view_count'},inplace=True)
df_board=df_board[['code', 'title', 'content', 'date', 'like', 'unlike','view_count']]

In [4]:
df_board.head()

,code,title,content,date,like,unlike,view_count
0,000270,공매도 이기고 가면 올라갈 수는 있음 ㅋ,어찌하냐?,2022.12.21 14:29,1,0,564
1,000270,기아 다시 9만 가즈아아아아,가즈아 가즈아 애국기아 민노 out,2022.12.21 14:20,5,2,664
2,000270,기관과 외국인 지들끼리 놀아라 하고,개미는 발빼자 개미할배는 발뺀지 오래다,2022.12.21 14:29,1,1,569
3,000270,"앞으로 사재기 없다, 공장 지을 때 참고 하세요.",it's this both of all i 'm very consider\nthe ...,2022.12.21 14:29,1,0,588
4,000270,외국인들,외국인들\n어제 사는 척 개미 꼬시기 해놓고\n오늘 거래량도 없이 매도하는 거 봐라...,2022.12.21 14:36,2,0,598


In [5]:
df_board.shape

(149210, 7)

In [ ]:
df_board.isnull().sum()

In [6]:
df_board.dropna(inplace=True)

In [7]:
df_board.shape

(149208, 7)

In [8]:
# 최소한의 반응이 있는 글들 사용
# 좋아요 싫어요 합이 1 이상
df_board = df_board.loc[(df_board['like']+df_board['unlike'])>=1]
df_board.reset_index(drop=True,inplace=True)
df_board.shape

(110964, 7)

In [9]:
df_board['date'] = df_board['date'].apply(lambda x: pd.to_datetime(x))
df_date = df_date['날짜']
df_date = df_date.apply(lambda x: pd.to_datetime(str(x)))

In [10]:
df_board['content'] = df_board['content'].apply(lambda x : x.replace('\n',''))

In [11]:
idx = np.where((df_board['date']>datetime.datetime(2017, 11, 30, 17, 50))&(df_board['date']<datetime.datetime(2022, 11, 30, 17,50)))

In [12]:
df_board = df_board.iloc[idx]
df_board.reset_index(drop=True,inplace=True)

In [13]:
idx1 = np.where((df_date>=datetime.datetime(2017, 12, 1))&(df_date<=datetime.datetime(2022, 11, 30)))

In [14]:
df_date = df_date[np.min(idx1)-1:np.max(idx1)+2]
df_date.reset_index(drop=True,inplace=True)


In [16]:
def date_category(x):
    for i in range(len(df_date)-1):
        if ((df_date[i] + datetime.timedelta(hours=17,minutes=50)) <= x) & (x<=(df_date[i+1] + datetime.timedelta(hours=17,minutes=49))):
            return df_date[i+1].strftime('%Y-%m-%d')

In [17]:
df_board['date_cate']= df_board['date'].apply(lambda x: date_category(x))

In [23]:
df_board.to_csv('./../Data/boardcomment/pre_comment/comment_kia.csv',encoding='utf-8-sig')

뉴스 date tagging

In [2]:
# 네이버 일단 제외"035420":"NAVER"
kospi_list = {"005380":'현대차',"068270":'셀트리온',"005930":'삼성전자',"000270":'기아',"373220":'LG에너지솔루션',"207940":'삼성바이오로직스',"000660":'SK하이닉스',"006400":'삼성SDI',"051910":"LG화학"}

In [ ]:
# 일별 시세 데이터프레임 불러오기
daily_list=[]
for i in os.listdir('./../Data/daily_price/kospi'):
    for j in kospi_list:
        if j in i:
            globals()[f'df_daily_{j}']= pd.read_csv('./../Data/daily_price/kospi/'+i)
            daily_list.append(f'df_daily_{j}')
            print(f'df_daily_{j}',i)

In [ ]:
# 뉴스 데이터프레임 불러오기
news_list = []
for i in os.listdir('./../Data/news'):
    for j in kospi_list:
        if kospi_list[j] in i:
            globals()[f'df_news_{j}']= pd.read_csv('./../Data/news/'+i)
            news_list.append(f'df_news_{j}')
            print(f'df_news_{j}',i)

In [ ]:
def date_category(x,df_date):
    for i in range(len(df_date)-1):
        if ((df_date['날짜'][i] + datetime.timedelta(hours=17,minutes=50)) <= x) & (x<=(df_date['날짜'][i+1] + datetime.timedelta(hours=17,minutes=49))):
            return df_date['날짜'][i+1].strftime('%Y-%m-%d')

In [ ]:
def merge_news(df_news,df_date,title_code):
    df_news['date_hour'] = df_news['date_hour'].apply(lambda x: pd.to_datetime(x))

    df_date['날짜'] = df_date['날짜'].apply(lambda x: pd.to_datetime(str(x)))

    idx = np.where((df_news['date_hour']>=datetime.datetime(2017, 11, 30, 17, 50))&(df_news['date_hour']<datetime.datetime(2022, 11, 30, 17,50)))
    df_news = df_news.iloc[idx]
    df_news.reset_index(drop=True,inplace=True)


    idx1 = np.where((df_date['날짜']>=datetime.datetime(2017, 12, 1))&(df_date['날짜']<datetime.datetime(2022, 11, 30)))


    if np.min(idx1) != 0:
        df_date = df_date[np.min(idx1)-1:np.max(idx1)+2]
        df_date.reset_index(drop=True,inplace=True)
    else:
        df_date = df_date[np.min(idx1):np.max(idx1)+2]
        df_date.reset_index(drop=True,inplace=True)


    df_news['date_cate']= df_news['date_hour'].apply(lambda x: date_category(x,df_date))

    df_date['전일비_비율'] = df_date['전일비_비율'].apply(lambda x : x.replace('%','')).astype(float)
    
    df_date['standard']=(df_date['전일비_비율']-np.mean(df_date['전일비_비율']))/np.std(df_date['전일비_비율'])
    df_date['standeard_tagging0.5'] = df_date['standard'].apply(lambda x: 0 if x<-1 else
                                                                        1 if x<-0.5 else
                                                                        2 if x<0.5 else
                                                                        3 if x<1 else
                                                                        4 )
    temp_df = df_date.loc[:,['날짜','전일비_비율','standard','standeard_tagging0.5']]
    temp_df['날짜'] = temp_df['날짜'].astype(str)

    result = pd.merge(df_news,temp_df, left_on ='date_cate',right_on='날짜')

    result.to_csv(f'./../Data/news_tagging/{str(title_code)}_tagging.csv',encoding='utf-8-sig',index=False)

In [ ]:
for i in news_list:
    for j in daily_list:
        if i[-6:] == j[-6:]:
            title_code = i[-6:]
            print(i,j,kospi_list[title_code])

In [ ]:
merge_news(df_news_373220, df_daily_373220, 'LG에너지솔루션')
merge_news(df_news_051910, df_daily_051910, 'LG화학')
merge_news(df_news_000660, df_daily_000660, 'SK하이닉스')
merge_news(df_news_000270, df_daily_000270, '기아')
merge_news(df_news_006400, df_daily_006400, '삼성SDI')
merge_news(df_news_207940, df_daily_207940, '삼성바이오로직스')
merge_news(df_news_005930, df_daily_005930, '삼성전자')
merge_news(df_news_068270, df_daily_068270, '셀트리온')
merge_news(df_news_005380, df_daily_005380, '현대차')